<a href="https://colab.research.google.com/github/juhumkwon/Data/blob/main/%EC%9E%90%EC%9C%A8%EC%A3%BC%ED%96%89(%EB%B3%B4%ED%96%89%EC%9E%90_%ED%96%89%EB%8F%99_%EC%98%88%EC%B8%A1).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# === 단일 셀: CNN + Transformer + Attention Score 출력 예제 ===
import tensorflow as tf
from tensorflow.keras import layers, Model
import numpy as np

# ---------------- 설정 ----------------
SEQ_LEN   = 32
IMG_SHAPE = (224, 224, 3)
EMBED_DIM = 512
NUM_HEADS = 2

# 1) 입력
video_input = tf.keras.Input((SEQ_LEN, *IMG_SHAPE), name="video")

# 2) CNN 백본 (MobileNetV2) → 프레임 feature
cnn = tf.keras.applications.MobileNetV2(include_top=False, pooling="avg")
cnn.trainable = False
x = layers.TimeDistributed(cnn)(video_input)          # (B, 32, 1280)
x = layers.Dense(EMBED_DIM)(x)                        # (B, 32, 512)

# 3) Positional Encoding
pos = tf.range(SEQ_LEN)
x += layers.Embedding(SEQ_LEN, EMBED_DIM)(pos)

# 4) Multi‑Head Self‑Attention (scores 반환)
attn_layer = layers.MultiHeadAttention(
    num_heads=NUM_HEADS, key_dim=64
)
attn_out = attn_layer(x, x)              # attn_scores: (B, H, 32, 32)

# 5) 분류 헤드 (crossing 예/아니오)
z = layers.GlobalAveragePooling1D()(attn_out)
z = layers.Dense(128, activation="relu")(z)
output = layers.Dense(1, activation="sigmoid")(z)

# 6) 모델
model = Model(video_input, output)
model.compile(loss="binary_crossentropy",
              optimizer=tf.keras.optimizers.Adam(1e-4))

# 7) 더미 시퀀스 → forward → Attention score 확인
dummy = np.random.rand(1, SEQ_LEN, *IMG_SHAPE).astype(np.float32)
pred = model(dummy)
print("예측 확률:", pred.numpy().squeeze())

/tmp/ipython-input-2-658772612.py:16: UserWarning: `input_shape` is undefined or non-square, or `rows` is not in [96, 128, 160, 192, 224]. Weights for input shape (224, 224) will be loaded as the default.
  cnn = tf.keras.applications.MobileNetV2(include_top=False, pooling="avg")


예측 확률: 0.51614964
